 # 【ラビットチャレンジ】深層学習後編（day3,day4）
 



 ## 1.再起型ニューラルネットワークの概念
 
時系列データに対応できるニューラルネットワーク

系列データとは？時間的順序を追って一定間隔ごとに観察され，しかも相互に統計的依存関係が認められるようなデータの系列
* 音声データ
* テキストデータ

<img src="RNN_1.jpg">
左図を展開したものが、右となる。

中間層への入力は、前回の中間層の状態 $ Z_{(t-1)} $ からの入力と新たな入力 $ X_t $ が加えられたものとなる。

計算式は以下の通り

 
 
 $$
 u^t = W_{(in)} x^t + W z^{t-1} + b
 $$
 
 $$
 z^t = f(W_{(in)}x^t + W z^{t-1}+b)
 $$
 
 $$
 v^t = W_{(out)}z^t + c
 $$
 
 $$
 y^t = g(W_{(out)}z^t+c)
 $$

計算には、
* 入力から現在の中間層を定義する際の重み $ W_{(in)} $, 
* 中間層から出力を定義する際の重み $ W_{(out)} $
* 中間層の前の状態から中間層を定義する際の重み $ W $ 

#### 確認テスト
の3つの重みがある
* 入力から現在の中間層を定義する際の重み $ W_{(in)} $, 
* 中間層から出力を定義する際の重み $ W_{(out)} $
* 中間層の前の状態から中間層を定義する際の重み $ W $ 


#### 実装演習

<b>順伝搬</b>

スライド S1)1-3
$$
u^t = \mathbf{W}_{in} x^t + \mathbf{W} z^{(t+1)} + b \\
z^t = f( \mathbf{W}_{in} x^t + \mathbf{W} z^{(t+1)} + b) \\
v^t = \mathbf{W}_{out} z^t + c \\
y^t = g(\mathbf{W}_{out} z^t + c)
$$

対応するコード。
 <code>
        u[:,t+1] = np.dot(X, W_in) + np.dot(z[:,t].reshape(1, -1), W)
        z[:,t+1] = functions.sigmoid(u[:,t+1])
        y[:,t] = functions.sigmoid(np.dot(z[:,t+1].reshape(1, -1), W_out))
 </code>
 
 ### BPTT (Back Propagation Through Time)
 

RNN における逆伝搬の計算式は以下のようになる。



$$
\frac{\partial E}{\partial W_{(in)}}=\frac{\partial E}{\partial u^t}\left[\frac{\partial u^t}{\partial W_{(in)}}\right]^T=\delta^t[x^t]^T \\
\frac{\partial E}{\partial W_{(out)}}=\frac{\partial E}{\partial v^t}\left[\frac{\partial v^t}{\partial W_{(out)}}\right]^T=\delta^{out,t}[z^t]^T \\
\frac{\partial E}{\partial W}=\frac{\partial E}{\partial u^t}\left[\frac{\partial u^t}{\partial W}\right]^T=\delta^t[z^{t-1}]^T \\
\frac{\partial E}{\partial b}=\frac{\partial E}{\partial u^t}\frac{\partial u^t}{\partial b} = \delta^t \\
\frac{\partial E}{\partial c}=\frac{\partial E}{\partial v^t}\frac{\partial v^t}{\partial c} = \delta^{out,t}
$$

重み W が3種類あるので、W に関する微分が3式、bに関する微分が 1 つ、cに関する式が1つ。


$ \delta^t $ は $ {\partial E \over \partial u} $、$ \delta^{out,t} $ は $ {\partial E \over \partial v} $を表す。


パラメータ更新式

$$
W^{t+1}_{(in)}=W^{t}_{(in)}-\epsilon\frac{\partial E}{\partial W_{(in)}}=W^t_{(in)}-\epsilon\sum_{z=0}^{T_t}\delta^{t-z}[x^{t-z}]^T
$$
$$
W^{t+1}_{(out)}=W^{t}_{(out)}-\epsilon\frac{\partial E}{\partial W_{(in)}}=W^t_{(out)}-\epsilon\delta^{out,t}[z^{t}]^T
$$
$$
W^{t+1}=W^{t}-\epsilon\frac{\partial E}{\partial W}=W^t_{(in)}-\epsilon\sum_{z=0}^{T_t}\delta^{t-z}[z^{t-z-1}]^T,\\
$$
$$
b^{t+1}=b^t−\epsilon \frac{\partial E}{\partial b}=b^t−\epsilon  \sum_{z=0}^{T_t} \delta^{t−z}
$$
$$
c^{t+1}=c^t−\epsilon\frac{\partial E}{\partial c} =c^t−\epsilon \delta^{out,t}
$$

#### 確認テスト
$$
y_1 = g(W_{out} s_1 + c) \\
s_1 = f(w_{in} x_1 + w s_0 + b )
$$


#### 実装演習　3_1_simple_RNN

<b>逆伝搬</b>

スライド S1)2-2
$$
\frac{\partial E}{\partial W_{(in)}}=\frac{\partial E}{\partial u^t}\left[\frac{\partial u^t}{\partial W_{(in)}}\right]^T=\delta^t[x^t]^T \\
\frac{\partial E}{\partial W_{(out)}}=\frac{\partial E}{\partial v^t}\left[\frac{\partial v^t}{\partial W_{(out)}}\right]^T=\delta^{out,t}[z^t]^T \\
\frac{\partial E}{\partial W}=\frac{\partial E}{\partial u^t}\left[\frac{\partial u^t}{\partial W}\right]^T=\delta^t[z^{t-1}]^T
$$

対応するコード
<code>
        # 勾配更新
        W_out_grad += np.dot(z[:,t+1].reshape(-1,1), delta_out[:,t].reshape(-1,1))
        W_grad += np.dot(z[:,t].reshape(-1,1), delta[:,t].reshape(1,-1))
        W_in_grad += np.dot(X.T, delta[:,t].reshape(1,-1))
</code>


 ## 2.LSTM(Long-short term model)

RNN の問題点。
活性化関数の微分により、勾配消失問題が発生する。


<img src="LSTM_1.png">

この図がわかりにくかったので、別の図を探した。

<img src="LSTM_2.jpg">
$$
f^{(t)} = f^\sigma_f(x^{(t)},h^{(t-1)}) \\
i^{(t)} = f^\sigma_i(x^{(t)},h^{(t-1)}) \\
o^{(t)} = f^\sigma_o(x^{(t)},h^{(t-1)}) \\
\tilde{c}^{(t)} =  f^{tanh}_{\tilde{c}}(x^{(t)},h^{(t-1)}) \\
c^{(t)} =  f^{(t)}\odot c^{(t-1)} + i^{(t)} \odot \tilde{c}^{(t)} \\
y^{(t)} = h^{(t)} = o^{(t)} \odot c^{(t)}
$$

シグモイド関数は、0～1の値をとり、どの程度の情報を流すかという割合を決定する意味でゲートの働きをする。

* $ i $ input gate vector
* $ o $ output gate vector 
* $ f $ forget gate vector
* $ c $ context vector : 長期記憶用
* $ h $ 短期記憶用

### CEC

CEC ( $ C^{t} $)は長期記憶のメモリで、この式は過去の情報と最新の情報をどの程度の割合で覚えさせるかを決定している。

$$
c^{(t)} =  f^{(t)}\odot c^{(t-1)} + i^{(t)} \odot \tilde{c}^{(t)} 
$$

#### 演習チャレンジ

上記のように、入力に対し、入力ゲートの値をかけたものと、ひとつ前のセルに対し忘却ゲートをかけたものを足し合わせる必要があることから、

(3）input_gate* a + forget_gate* c

が正解

### 覗き穴結合

LSTM のバリエーションとして、CEC  から、出力ゲート、入力ゲートに「覗き穴結合」を行うものが存在する。
入出力をCECの現在の値を反映させる目的であるが、あまり効果がなかった。




 ## 3.GRU (Gated Recurrent Unit)
 
 従来のLSTMでは、パラメータが多数存在していたため、計算負荷が大きかった。しかし、GRUでは、そのパラメータを大幅に削減し、精度は同等またはそれ以上が望める様になった構造。
 
 <img src='GRU_1.jpg'>
 
$$
r(t)=W_r x(t) + U_r \cdot h(t-1) + b_h(t) \\
z(t) = W_z x(t) + U_z \cdot h(t-1) + b_z(t) \\
\bar{h}(t) = f(W_r x(t) + U_r \cdot (r(t) \cdot h(t-1))) b_h (t)) \\
h(t)=z(t) \cdot h(t-1) + (1-z(t))\cdot \bar{h}(t)
$$
 
* r : どのくらい忘れるか
* z : どのくらい更新するか

を制御している



やっぱりわかりにくいので別の図
<img src='GRU_2.jpg'>
$$
z^{(t)} = f^\sigma_z(x^{(t)},h^{(t-1)}) \\
r^{(t)} = f^\sigma_r(x^{(t)},h^{(t-1)}) \\
\tilde{h}^{(t)} =  f^{tanh}_{\tilde{h}}(x^{(t)},r^{(t)} \odot h^{(t-1)}) \\
h^{(t)} = (1-z^{(t-1)})\odot h^{(t-1)} + z^{(t)} \odot \tilde{h}^{(t)}
$$
* r : reset gate vector
* z : update gate vector

 
 #### 確認テスト
 <b>LSTMとGRUの違い</b>
 
 GRU は LSTM と比較して、
 
 *  入力ゲート、出力ゲート、忘却ゲートがなくなり、リセットゲート、更新ゲートを追加
 * CECが存在しない
 
 #### 実習
 
 (3_3_predict_sin)
 
 <img src='RNN_2.jpg'>

 ## 4.双方向RNN
  過去の情報だけでなく、未来の情報を加味することで、精度を向上させるためのモデル
  <img src="BIDIRECTIONAL_1.jpg">
  
  過去の情報 Sと未来の情報 S' をhidden state として保持する。
  
  
 
 #### 演習チャレンジ

双方向RNNでは、順方向と逆方向の表現を時間方向に連結したものを記憶する必要があるので、
<code>
np.concatenate([h_f, h_b[::-1]], axis=1)
</code>
が正解
  

 ## 5.Seq2Seq
 
 <img src="SEQ2SEQ_1.png">
 
* 機械対話や、機械翻訳などに使用されるモデル
* Encoder-Decoderモデルの一種。RNNで構成されたエンコーダーとデコーダをつなぐ。
* エンコーダーで文章を通すと、隠れ層には単語の意味がベクトル表現で保存される。one-hot ベクトルを意味ベクトル(embeded vector)に変換。単語の意味が似ている enbeded vector 表現でも距離が近い
* デコーダーでは意味ベクトルのシーケンスから、単語列を生成する。
* 単語/意味ベクトルの変換で異なる言語のものを用いることで翻訳が可能となる。

#### 確認テスト
3.seq2seq は encoder-decoderモデルの一種で機械翻訳等で用いられる。
#### 演習チャレンジ
単語を enbeding 表現に変換んする必要があるので、$ Ew $の計算が必要

(1) E.dot(w)

### HRED
* 自動会話
* 過去 n-1 個の会話から次の会話を生成する
* 隠れ層を RNN 構造にすることにより、文脈を覚える Context RNN

問題点
* ありがちな答えしかださなくなる
* 短い回答のみ


### VHRED
* HRED に VAE の潜在変数の概念を追加することで解決した構造

### VAE
* 潜在変数

#### オートエンコーダー
* 教師なし学習の一種
* 入力画像を圧縮・伸縮（エンコード・デコード）を行った出力が、元の画像と同じになるようにする。
*　次元削減が可能(潜在変数 Zで表す）
### VAE
* 通常のオートエンコーダーの場合、潜在変数 z に押し込めるが、zの構造がわからに
* VAE では、z の正則化を行い、$ z \sim N(0,1) $ の確率分布を仮定したもの。
* 元データが近いものは、近いデータにエンコーディングされるようにしたい
* VAE にノイズを加えることにより、汎用性の高い答えをだすことができる。

#### 確認テスト
VAE は潜在変数に確率分布を導入したもの

#### 確認テスト

* seq2seq とはある時系列データからある時系列データを生成するもの

なので(2)が正解。

尚、
* HRED 文脈の意味を解釈しながら、時系列データを生成する
* VHRED HREDを改良したもので、VAE を利用することにより、より自然な会話ができるように改良したもの

#### 園主チャレンジ


#### Seq2seqのコード確認

lecture_chap1_exercise_public.ipynbから

##### Encoder 
 GRU を使って入力を意味ベクトルに変換

<code>
    def forward(self, seqs, input_lengths, hidden=None):
        """
        :param seqs: tensor, 入力のバッチ, size=(max_length, batch_size)
        :param input_lengths: 入力のバッチの各サンプルの文長
        :param hidden: tensor, 隠れ状態の初期値, Noneの場合は0で初期化される
        :return output: tensor, Encoderの出力, size=(max_length, batch_size, hidden_size)
        :return hidden: tensor, Encoderの隠れ状態, size=(1, batch_size, hidden_size)
        """
        emb = self.embedding(seqs) # seqsはパディング済み
        packed = pack_padded_sequence(emb, input_lengths) # PackedSequenceオブジェクトに変換
        output, hidden = self.gru(packed, hidden)
        output, _ = pad_packed_sequence(output)
        return output, hidden
</code>

##### Decoder
 GRUを用いて、出力単語 $y_i$の確率分布を計算する。
 
 $$
 p(y_i)=p(y_i|y_{(i-1)}, S^{(d)}_i, C)
 $$
 
 $y_i$ 前の単語、$ S^{(d)}_i$隠れ状態、$C$意味ベクトル
 
出力は $ P(y_i) $というｙ、単語の確率として出力される。


<code>
    def forward(self, seqs, hidden):
        """
        :param seqs: tensor, 入力のバッチ, size=(1, batch_size)
        :param hidden: tensor, 隠れ状態の初期値, Noneの場合は0で初期化される
        :return output: tensor, Decoderの出力, size=(1, batch_size, output_size)
        :return hidden: tensor, Decoderの隠れ状態, size=(1, batch_size, hidden_size)
        """
        emb = self.embedding(seqs)
        output, hidden = self.gru(emb, hidden)
        output = self.out(output)
        return output, hidden
</code>

Encoder-Decoder をつなぐ

<code>
    def forward(self, batch_X, lengths_X, max_length, batch_Y=None, use_teacher_forcing=False):
        """
        :param batch_X: tensor, 入力系列のバッチ, size=(max_length, batch_size)
        :param lengths_X: list, 入力系列のバッチ内の各サンプルの文長
        :param max_length: int, Decoderの最大文長
        :param batch_Y: tensor, Decoderで用いるターゲット系列
        :param use_teacher_forcing: Decoderでターゲット系列を入力とするフラグ
        :return decoder_outputs: tensor, Decoderの出力, 
            size=(max_length, batch_size, self.decoder.output_size)
        """
        _, encoder_hidden = self.encoder(batch_X, lengths_X)
        # decoderの入力と隠れ層の初期状態を定義
        decoder_input = torch.tensor([BOS] * self.beam_size, dtype=torch.long, device=device)
        decoder_input = decoder_input.unsqueeze(0)  # (1, batch_size)
        decoder_hidden = encoder_hidden
        # beam_sizeの数だけrepeatする
        decoder_input = decoder_input.expand(1, beam_size)
        decoder_hidden = decoder_hidden.expand(1, beam_size, -1).contiguous()
        # decoderの出力のホルダーを定義
        decoder_outputs = torch.zeros(max_length, _batch_size, self.decoder.output_size, device=device) # max_length分の固定長
        # 各時刻ごとに処理
        for t in range(max_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            decoder_outputs[t] = decoder_output
            # 次の時刻のdecoderの入力を決定
            if use_teacher_forcing and batch_Y is not None:  # teacher forceの場合、ターゲット系列を用いる
                decoder_input = batch_Y[t].unsqueeze(0)
            else:  # teacher forceでない場合、自身の出力を用いる
                decoder_input = decoder_output.max(-1)[1]
        return decoder_outputs
</code>

ここでは、decoder_output.max(-1)[1]で最も確率の高い単語を単純に選んでいるが、実際の単語を選択する際には Beam Search という手法をとる。




 ## 6.Word2vec
 
 単語のベクトル表現を行う場合、one-hot ベクトル表現をそのまま使うと、データ量が巨大になる（単語数分の要素が必要）
 
 $$
 'apple' \rightarrow [0,0,\dots, 1, 0, \dots ] \\
  'pear' \rightarrow [0,0,\dots, 0, 1, \dots ]
 $$
 
 
そこで、one-hotベクトル表現で表現されているベクトルを、低次元の空間にマッピングし、
また、意味的に近い単語を近い位置になるような変換を行うのが word2vec
 
この変換行列は COW または skip-gram という学習を行わせることで
間接的に取得することができる。

### COW 
前後の単語列から、中央の単語を予測するモデル

### skip-gram
一つの単語から、前後の単語を推測するモデル

COW 等を学習させると、意味ベクトルから元のワンホットベクトル表現への変換行列を得ることが
できるが、これがまさに単語の辞書に相当する

### ベクトル表現

word2vec によって単語のベクトル表現を行うことにより、
$$
'King' - 'man' + 'woman' = 'Queen'
$$
等といった、単語の意味的な演算を行うことが可能となる。







 ## 7.Attention Mechanism
 
 入力と出力のどの単語が関連しているのかの関連度を学習する仕組み。
 
1文の中で重要な単語を自力で見つけられるようにする仕組みとなる。

現在では様々なネットワークで利用される。

* Transformer
* BERT
* GPT-3

等でも使われている。

機械翻訳で使われている

Encoder-Decoder の限界

ベクトルのサイズが固定で、扱える情報の量や複雑さに制限

Encoder 部分は GRU

単語を意味ベクトルに変換

Decoder
$$
P(y_i)=p(y_i|y_{i-1}, S^{(d)}_i, C)
$$

* $ y_i $ : i番目の単語
* $ s^{(d)}_i $: 隠れ状態
* $ c $ : 意味ベクトル


### 確認テスト 
* RNN：時系列データを処理するのに適したネットワーク
* Word2Vec：単語の分散表現を得る手法
* seq2seq：一つの時系列データから別の時系列データを得るネットワーク
* Attention：時系列データの中身に対して関連性の重みをつける手法

 ## 8.強化学習
 長期的に報酬を最大化できるように環境のなかで行動を選択できるエージェントを作ることを目標とする機械学習の一分野
 
行動の結果として与えられる利益(報酬)をもとに、行動を決定する原理を改善していく仕組み
 
 <img src="REINFORCEMENT_1.jpg">
 
### 方策
ある状態において行える行動の中から何を選ぶかの確率を表したもの。

状態sにおいて、行動aを起こした時の方策
$$
\pi_t(a|s)
$$

### 報酬

ある行動をとった時に得られるもの

$$
R_t
$$

### 収益

ある時点からみて、最後までやり通した時の報酬の合計

過去の収益に対しては割引率 $ \gamma $ がかかり、

$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+2} + \dots + = \sum_k^\infty \gamma^k R_{r+t+1}
$$

### 状態価値関数
状態 s において、方策 $ \pi $ に従った時の期待収益

$$
V^\pi(s) = \mathbf{E}_\pi [G_t|S_t=s]
$$


### 行動価値関数
ある状態 s において行動 a をとり、その後方策$\pi$従った場合の期待収益

$$
Q^\pi(s,a) = \mathbf{E}_\pi [G_t|S_t=s, A_t=a]
$$

すなわち、
$$
V^\pi(s) = \sum_\pi \pi(a|s)Q^\pi(s,a)
$$


### 強化学習の手法

方策ベースと価値ベース

*方策ベース：明示的な方策があり、現在の方策を改善する
*価値ベース：明示的な方策はなく、サンプルから学習し、最適な価値関数の値を逐次推定する



#### 動的計画法
環境の完全なモデルがマルコフ決定勝てとして与えられている時に提供できるモデル。

#### モンテカルロ法
遷移のサンプルを取得し、与えられた収益を平均化することによって行動価値を推定
価値関数や方策の改善は、一連の行動（エピソード）終了後に行う。

#### TD学習
目標の価値と現在の価値のずれを修正することによって、価値関数を推定する。

Sarsa と Q 学習がある。

Sarsa
$$
Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha [R_{(t+1)} + \gamma Q(S_{(t+1)}, A_{(t+1)}) - Q(S_t, A_t)] 
$$

Q学習
$$
Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha [R_{(t+1)} + \gamma \max_\alpha Q(S_{(t+1)},\alpha) - Q(S_t, A_t)] 
$$

#### 方策勾配法
方策$ \pi $がパラメタ$\theta$をつかって
$$
\pi(a|s,\theta)
$$
であらわされるとする。この時、勾配法により最適な$\theta$を求める。
$$
\theta^{(t+1)} = \theta^{(t)}+\eta \nabla J(\theta)
$$
方策勾配定理により、
$$
\nabla_\theta J(\theta)=\mathbf{E}_{\pi_\alpha}[(\nabla_\theta \log \pi_\theta(a|s)Q^\pi(s,a))]
$$
が成り立つ。


 ## 9.AlpahaGO
 
 強化学習を利用した碁の囲碁プログラム
 
 現在、
 
 * AlphagoGO Lee
 * AlphaGO ZERO
 
 の2種類がある。
 
 ### AlphaGO Lee
 
方策関数を表現するPolicyNetと価値関数を表現するValueNet 
の2つのネットワークがあり、
二つとも畳み込みネットワークで構成さえれる。

PolicyNet は 19x19 マスの着手予想確率が出力され、
ValueNet は現局面の勝率が 1 ～　-1で出力される。


学習は、

1. 教師あり学習によるRollOutPolicyとPolicyNetの学習
2. 強化学習によるPolicyNetの学習
3. 強化学習によるValueNetの学習

のステップで行われる。

1. 教師あり学習によるRollOutPolicyとPolicyNetの学習

KGS Go Server（ネット囲碁対局サイト）の棋譜データから3000万局面分の教師を用意し、教師と同じ着手を予測できるよう学習を行った。

2. 強化学習によるPolicyNetの学習

現状のPolicyNetとPolicyPoolからランダムに選択されたPolicyNetと対局シミュレーションを行い、その結果を用いて方策勾配法で学習を行った。

3. 強化学習によるValueNetの学習

PolicyNetを使用して対局シミュレーションを行い、その結果の勝敗を教師として学習した。


### AlphaGO ZERO


AlphaGo(Lee) との違い

* 教師あり学習を一切行わず、強化学習のみで作成
* 特徴入力からヒューリスティックな要素を排除し、石の配置のみにした
* PolicyNetとValueNetを１つのネットワークに統合した
* Residual Netを導入した
* モンテカルロ木探索からRollOutシミュレーションをなくした


#### ResidualNetwork

ネットワークにショートカットを設ける
勾配消失問題、勾配爆発を避ける

また、
アンサンブル効果が期待できる

100層を超えるネットワークでも安定するようになった。（AlphaGO ZERO では ResidualBlock が39回）






## 10.軽量化・高速化技術
 
深層学習は多くのデータを使用したり、パラメータ調整のために多くの時間を使用したりするため、高速な計算が求められる
そのため、以下のような高速、軽量化の手法が用いられる。

### 分散深層学習
 * データ並列化：データを分割し、複数のワーカーで計算を行う。
 * モデル並列化：モデルを各ワーカに分割し、それぞれのモデルで計算させる
 縦方向、横方向に分割
 
### GPUによる高速化
 * GPGPUの利用
 
 * CPU
高性能なコアが少数

複雑で連続的な処理が得意
 * GPU
 低性能なコアが多数。簡単な並列処理が得意
 
 GPUを使うためのフレームワークは主に2種類。ただし、 ほとんど CUDAでの実装 
 * CUDA
 * OpenCL
 


 
### モデルの軽量化
 * 量子化　: 64bitの浮動小数を 32bit に精度を落とす等
 * 蒸留　：規模の大きなモデルの知識を使い軽量なモデルの作成を行う。
教師モデル、生徒モデルを作成

 * プルーニング　：　モデルの精度に寄与が少ないニューロンを削減する
 重みが小さいところを削除する
 

 ## 11.応用モデル
 
 ### mobilenet

畳み込みを工夫

Depthwise ConvolutionとPointwise Convolutionの組み合わせで軽量化を実現

#### Depthwise

一枚のフィルタで全チャンネルの畳み込みを行う。

計算量
* 一般的な畳み込みの計算量 : H x W x K x K x C x M
* DepthWize  : H x W x K x K x C


#### PointWise

畳み込みをdepthwise, pointwise に分離
1X1　のフィルタを用いる

計算量 H x W x C x M

### densenet
画像認識ネットワーク

### DenseNet

### Batch Normalization

* レイヤー間を流れるデータの分布を、ミニバッチ単位で平均が0・分散が1になるように正規化する。
* Batch Normalizationはニューラルネットワークにおいて学習時間の短縮や初期値への依存低減、過学習の抑制など効果がある


### Wavenet

* 生の音声波形を生成するモデル

*　時系列データに対して畳み込み　（Dilated Convolution)を適用する。

*層が深くなるにつれて畳み込むリンクを離す
*受容野を簡単に増やすことができる

<img src='DILATED_1.jpg'>


 ## 12.Transformer
  Attention 機構のみを使った計算量の少ないモデル。

### Encoder / Decoder モデルの問題点
 seq2seq 等、Encoder / Decoer を使ったモデルでは、固定長ベクトルを使うため、長い文章でのスコアがよくなかった。

 ### Attention

 翻訳先の各単語を選択する際に、翻訳元の文中の各単語の隠れ状態を利用
 


翻訳元の各単語の隠れ状態を利用する。

$$
p(y_i) = p(y_i|p_{i-1}, S^{(d)}_i, C_i) \\
C_i = \sum_j \alpha_{ij} h_j \\
\alpha_{ij} = {exp(e_ij) \over \sum_k exp(e_{ik})}
$$
$ C_i $は Attention で $h_j$の重み付き和。$\alpha_i$の総和は1


Attention は query(検索クエリ)に一致するkeyを索引し、対応するvalueを取り出す操作と解釈できる。

 $$
 \sigma(Q, \mathbf{K}) = V
 $$
 
 元々機械翻訳で出てきたアルゴリズムであるが、多くのモデル（画像認識の ViT等)に適用されている。
 
 
 
 ### Transformerの構造
  
 <img src='TRANSFORMER_1.jpg'>
 
* RNN を使用していないため、軽い
* BERT, GPT-3 等にもTransfomer の技術が利用されている。
 
 
 
4  つの主要なモジュールからなる。

- Positional Encoding: 入出力の単語のEmbedding時に単語の位置情報を埋め込む
- Scaled Dot-Product Attention: 内積でAttentionを計算し、スケーリングを行う
- Multi-head Attention: Scaled Dot-Product Attentionを複数のヘッドで並列化する
- Position-Wise Feed Forward Network: 単語列の位置ごとに独立して処理を行う

### Position Encoding

TransformerはRNNを使用しておらず、単語順序を考慮することができないので、
入力データに位置情報を埋め込むPositional Encodingを加算する。


$$
PE_{(pos, 2i)} = \sin(pos/10000^{2i/d_{model}}) \\
PE_{(pos, 2i+1)} = \cos(pos/10000^{2i/d_{model}})
$$

$pos$は単語の位置、$i$は成分の次元


<code>
def position_encoding_init(n_position, d_pos_vec):
    """
    Positional Encodingのための行列の初期化を行う
    :param n_position: int, 系列長
    :param d_pos_vec: int, 隠れ層の次元数
    :return torch.tensor, size=(n_position, d_pos_vec)
    """
    # PADがある単語の位置はpos=0にしておき、position_encも0にする
    position_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / d_pos_vec) for j in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return torch.tensor(position_enc, dtype=torch.float)
</code>


### Scaled Dot-Product Attention

Attention には
* Additive Attention
* Dot-Product Attention
の2種類が存在するが、Dot-Product Attention の方がパラメタが少なく高速。Transformer で採用

Transfomer では、ベクトルの次元数、$d_k$が大きく言時、ない席が大きくなりすぎ、Softmax の勾配が極端に小さくなることを伏せくため、query($Q$)とkey($K$)の内積をスケーリング因子 $\sqrt{d_k}$ で除算する。

$$
Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V
$$ 

### Multi-head Attention

TransformerではAttentionを複数のヘッドで並列に行うMulti-Head Attentionを使用している。

複数のヘッドでAttentionを行うことにより、各ヘッドが異なる部分空間を処理でき、精度が向上する


### Position-wise Feed-Forward Networks

$$
FFN(x)=max(0,xW1,b1)W2+b2
$$

#### 補）評価指標

機械翻訳の精度の指標として BLEU がよく使われる。

BLEUは
 $$
 BLEU=\exp(min(0, 1 - \frac{r}{c}))\exp (\sum w_n \log P_n)
 $$
と定義される。

$P_n$はn-gramにおけるPrecisionで
 $$
Precision = { 参考訳に含まれる単語数 \over 提案訳文の単語数 }
 $$
と定義される。ただし、参考訳で現れる単語は一回のみ

 $$
\exp (\sum w_n \log P_n)
 $$
は 1～Nグラムまでの相乗平均をとることを意味する。

$ min(0, 1 - \frac{r}{c})$
は短い文におけるペナルティ(Brevity Penalty)で、
極端に短い文の方が上記で定義されるPrecisiohnが大きくなることへのペナルティである。

 

 
 

 ## 13.物体検知・セグメンテーション
 
物体検知では、入力画像に対して、どこに何を見つけたかを出力する。
出力項目としては、以下
* Bounding Box
* ラベル
* コンフィデンス

代表的なデータセットとして以下のものがある。
* VOC12
* ILSVRC17
* MS COCO18
* OICOD18
等

データ量が大きいだけではだめで、目的に合わせて選択する必要がある。

* クラス数
* 訓練データ＋テストデータ数
* BOX/画像

等

 
 
### 物体検知の評価指標


Accuracy だけでは、ダメ。

$$
Precision = { TP \over TP + FP}
$$
$$
Recall = {TP \over TP + FN}
$$

Recall =
クラスラベルだけでなく, 物体位置の予測精度も評価したい！クラスラベルだけでなく, 物体位置の予測精度も評価したい！

$$
IoU={TP \over TP+FP+FN}={Area\ of\ Overlap \over Area\ of\ Union}
$$

AP (Average Precision)

$$
AP = \int^1_0 P(R)dR
$$
PR曲線の下側の面積

クラス毎に計算。

$$
mAP = \frac{1}{C}\sum_C AP_c
$$

全クラスでの平均


SHFT から DCNN

代表的なネットワーク

|  年  |  ネットワーク  |
| ---- | ---- |
|  2012  |  AlexNet  |
|  2014  |  VGGNet, GoogLeNet  |
|  2015|  ResNet  |
|  2016|  DenseNet、Inception-ResNet  |
|  2017|  MobileNet  |
| 2018 | AmoebaNet |

代表的な物体検知フレームワーク

|  年  |  ネットワーク  |
| ---- | ---- |
|  2012  |  DCNN  |
|  2013  |  DecoNet, RCNN  |
|  2014  |  SPPNet  |
|  2015  |  Fast RCNN、Faster RCNN、YOLO、SSD  |
|  2016  |  RFCN、YOLO9000、FPN  |
|  2017  |  RetinaNet、Mask RCNN |
|  2018  |  CornerNet |


慮域の検出と、クラス判別を2段階で行うか、同時に行うかで

2段階検出/ 1段階検出

に分けられる。

### SSD (Single Shot Detector)

* DefaultBOX を用意：学習が進む毎に変形

1つの特徴量に対して複数の DefaultBOX を置く


マルチスケール特徴マップ

### Semantic Segmentation

各ピクセルに対してクラスを決定する。

ところが、
Covolution と Pooling を繰り返すことにより、解像度が落ちることが問題となる。→アップサンプリングが必要

Pooling の必要性→受容野を広げる

#### Decomvolution

<img src="DECONVOLUTION_1.jpg">

#### Unpooling
場所情報を Switch Variable として覚えておく

#### Up-Sampling

Poolingにより輪郭情報が失われ行く。
低レイヤーPooling層の出力をelement-wise addition することでローカルな情報を補完してからUp-samplin

#### Dilated Convolution

Covolution の段階で受容野を広げる





